In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report 
import re
import string


In [8]:
data_fake = pd.read_csv("C:/Users/Sachin Upadhyay/Downloads/DataSet_Misinfo_FAKE.csv")
data_true = pd.read_csv("C:/Users/Sachin Upadhyay/Downloads/DataSet_Misinfo_TRUE.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 152703: invalid start byte

In [4]:
data_true.head()

,Unnamed: 0,text
0,0,The head of a conservative Republican faction ...
1,1,Transgender people will be allowed for the fir...
2,2,The special counsel investigation of links bet...
3,3,Trump campaign adviser George Papadopoulos tol...
4,4,President Donald Trump called on the U.S. Post...


In [5]:
data_fake.head()

,Unnamed: 0,text
0,0,Donald Trump just couldn t wish all Americans ...
1,1,House Intelligence Committee Chairman Devin Nu...
2,2,"On Friday, it was revealed that former Milwauk..."
3,3,"On Christmas day, Donald Trump announced that ..."
4,4,Pope Francis used his annual Christmas Day mes...


In [6]:
data_fake["class"] = 0
data_true["class"] = 1

In [7]:
data_fake.shape , data_true.shape

((43642, 3), (34975, 3))

In [8]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head()

,Unnamed: 0,text,class
0,0,Donald Trump just couldn t wish all Americans ...,0
1,1,House Intelligence Committee Chairman Devin Nu...,0
2,2,"On Friday, it was revealed that former Milwauk...",0
3,3,"On Christmas day, Donald Trump announced that ...",0
4,4,Pope Francis used his annual Christmas Day mes...,0


In [9]:
data_merge.columns

Index(['Unnamed: 0', 'text', 'class'], dtype='object')

In [10]:
#Ramdom Shuffling of the data
data = data_merge.sample(frac = 1)

In [11]:
data = data.drop(data.columns[0], axis=1)
data.head()

,text,class
26568,The body of the Iraqi prisoner was found naked...,1
33307,President Enrique Peña Nieto has invited both ...,1
28539,Climate change has increased the likelihood of...,1
10752,Republican presidential front-runner Donald Tr...,1
27563,(Want to get this briefing by email? Here’s th...,1


In [12]:
#Removing the index column
data.reset_index(inplace= True)
data.drop(['index'],axis=1,inplace= True)

In [13]:
data.columns
data.head()

,text,class
0,The body of the Iraqi prisoner was found naked...,1
1,President Enrique Peña Nieto has invited both ...,1
2,Climate change has increased the likelihood of...,1
3,Republican presidential front-runner Donald Tr...,1
4,(Want to get this briefing by email? Here’s th...,1


In [14]:
def  textCleaner(text):
    text=str(text)
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub("\\W"," ",text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    return text

In [15]:
data['text'] = data['text'].apply(textCleaner)

In [16]:
x = data['text']
y = data['class']

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [19]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test  = vectorization.transform(x_test)

In [20]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [21]:
pred_lr = LR.predict(xv_test)
LR.score(xv_test,y_test)

0.939608242177563

In [22]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95     10951
           1       0.94      0.92      0.93      8704

    accuracy                           0.94     19655
   macro avg       0.94      0.94      0.94     19655
weighted avg       0.94      0.94      0.94     19655



In [23]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [24]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test,y_test)


0.8910200966675146

In [25]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90     10951
           1       0.89      0.86      0.87      8704

    accuracy                           0.89     19655
   macro avg       0.89      0.89      0.89     19655
weighted avg       0.89      0.89      0.89     19655



In [26]:
#manual Testing of the data

def output(n):
    if n==0:
        return "Alert !! Fake news detected"
    elif n==1:
        return "Not a fake News"

def manual_testing(news):
     testing_news = {"text":[news]}
     new_def_test = pd.DataFrame(testing_news)
     new_def_test['text'] = new_def_test['text'].apply(textCleaner)
     new_xv_test = new_def_test['text']
     print(new_xv_test)
     new_xv_test = vectorization.transform(new_xv_test)
     pred_LR= LR.predict(new_xv_test)
     pred_DT= DT.predict(new_xv_test)
        
     return print("\n\n LR prediction: {} \n DT Prediction: {} ".format(output(pred_LR[0]),output(pred_DT[0])))

In [27]:
news = str(input())
manual_testing(news)

WASHINGTON (Reuters) - The United States plans to spend $3.2 billion to develop antiviral pills for COVID-19 and other viruses that could become pandemics, the Biden administration said on Thursday. The plan is part of President Joe Biden’s broader effort to prepare for future pandemics by creating domestic production capabilities, improving detection and response and accelerating research and development. The administration said the pills, which could be taken at home to minimize serious illness and hospitalizations, would likely be available by the end of the year, subject to the completion of clinical trials and regulatory authorization. The pills would be designed to treat people already infected with the virus or at high risk of getting infected. “New antivirals that prevent serious COVID-19 illness and death, especially oral drugs that could be taken at home early in the course of disease, would be powerful tools for battling the pandemic and saving lives,” National Institute of 

In [28]:
#saving the model
import pickle
filename= 'finalized_model.pkl'
pickle.dump(LR,open(filename,'wb'))
